## Deliverable 2. Create a Customer Travel Destinations Map.

In [40]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [8]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv(r"/Users/ahuynh/Desktop/Bootcamp/Analysis Projects/Module_6/World_Weather_Analysis/Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Cherskiy,RU,68.7500,161.3000,20.17,96,93,3.02,overcast clouds
1,1,Saint George,US,37.1041,-113.5841,90.19,13,1,3.00,clear sky
2,2,Hobart,AU,-42.8794,147.3294,61.02,85,75,1.99,broken clouds
3,3,Vaini,TO,-21.2000,-175.2000,75.36,83,90,13.80,overcast clouds
4,4,Ushuaia,AR,-54.8000,-68.3000,38.86,70,75,14.97,light intensity drizzle


In [9]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 90


In [10]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Hobart,AU,-42.8794,147.3294,61.02,85,75,1.99,broken clouds
3,3,Vaini,TO,-21.2000,-175.2000,75.36,83,90,13.80,overcast clouds
5,5,Mogadishu,SO,2.0371,45.3438,78.12,84,40,13.33,scattered clouds
6,6,Albany,US,42.6001,-73.9662,62.67,93,100,1.21,light rain
7,7,Mount Isa,AU,-20.7333,139.5000,80.37,18,21,12.66,few clouds
8,8,Vila Velha,BR,-20.3297,-40.2925,78.75,78,40,16.11,scattered clouds
10,10,Hasaki,JP,35.7333,140.8333,72.00,75,0,10.38,clear sky
17,17,Atuona,PF,-9.8000,-139.0333,77.70,76,12,16.44,light rain
18,18,Kingsport,US,36.5484,-82.5618,70.12,96,90,1.99,moderate rain
19,19,Faanui,PF,-16.4833,-151.7500,78.91,70,42,9.44,scattered clouds


In [11]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                397
City                   397
Country                397
Lat                    397
Lng                    397
Max Temp               397
Humidity               397
Cloudiness             397
Wind Speed             397
Current Description    397
dtype: int64

In [12]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [13]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Hobart,AU,61.02,broken clouds,-42.8794,147.3294,
3,Vaini,TO,75.36,overcast clouds,-21.2000,-175.2000,
5,Mogadishu,SO,78.12,scattered clouds,2.0371,45.3438,
6,Albany,US,62.67,light rain,42.6001,-73.9662,
7,Mount Isa,AU,80.37,few clouds,-20.7333,139.5000,
8,Vila Velha,BR,78.75,scattered clouds,-20.3297,-40.2925,
10,Hasaki,JP,72.00,clear sky,35.7333,140.8333,
17,Atuona,PF,77.70,light rain,-9.8000,-139.0333,
18,Kingsport,US,70.12,moderate rain,36.5484,-82.5618,
19,Faanui,PF,78.91,scattered clouds,-16.4833,-151.7500,


In [14]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
        hotel

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [43]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Hobart,AU,61.02,broken clouds,-42.8794,147.3294,Mantra on Collins Hobart
3,Vaini,TO,75.36,overcast clouds,-21.2000,-175.2000,Keleti Beach Resort
5,Mogadishu,SO,78.12,scattered clouds,2.0371,45.3438,Hotel Juba
7,Mount Isa,AU,80.37,few clouds,-20.7333,139.5000,ibis Styles Mt Isa Verona
8,Vila Velha,BR,78.75,scattered clouds,-20.3297,-40.2925,Quality Suites Vila Velha
10,Hasaki,JP,72.00,clear sky,35.7333,140.8333,Hotel Sunrise Choshi
17,Atuona,PF,77.70,light rain,-9.8000,-139.0333,Villa Enata
18,Kingsport,US,70.12,moderate rain,36.5484,-82.5618,MeadowView Conference Resort & Convention Center
19,Faanui,PF,78.91,scattered clouds,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
20,Carlyle,US,73.80,broken clouds,38.6103,-89.3726,Mariner's Village at Carlyle Lake


In [46]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [49]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [50]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))